In [2]:
import requests
from sqlalchemy import URL, engine

from src.config import Settings

In [3]:
settings = Settings()

database = []

response = requests.get(
    url='http://127.0.0.1:8000/postgres/databases/6909b78f41dca376afd84862'
)
list_database = response.json().get('databases', [])

In [4]:
list_database

[{'database_name': 'brazilian_e_commerce',
  'table_list': [{'table_name': 'olist_customers_dataset',
    'original_filename': 'olist_customers_dataset.csv',
    'row_count': 99441,
    'column_count': 6,
    'columns': ['id',
     'customer_id',
     'customer_unique_id',
     'customer_zip_code_prefix',
     'customer_city',
     'customer_state'],
    'created_at': 1763782535,
    'updated_at': 1763800064},
   {'table_name': 'olist_order_items_dataset',
    'original_filename': 'olist_order_items_dataset.csv',
    'row_count': 112650,
    'column_count': 8,
    'columns': ['id',
     'order_id',
     'order_item_id',
     'product_id',
     'seller_id',
     'shipping_limit_date',
     'price',
     'freight_value'],
    'created_at': 1763782894,
    'updated_at': 1763800064},
   {'table_name': 'olist_order_payments_dataset',
    'original_filename': 'olist_order_payments_dataset.csv',
    'row_count': 103886,
    'column_count': 6,
    'columns': ['id',
     'order_id',
     'payme

In [ ]:
url = URL.create(
    drivername=settings.postgres_db.driver_name,
    username=settings.postgres_db.username,
    password=settings.postgres_db.password,
    host=settings.postgres_db.host, port=settings.postgres_db.port,
    database=self.
)